In [ ]:
import numpy as np
import random
import pandas as pd
import time
import datetime
from sklearn.model_selection import train_test_split

In [ ]:
#!pip install transformers
!pip install -U adapter-transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 108.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 20.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
train_data = pd.read_csv('DataAugmentation.csv')

In [ ]:
train_data = train_data[['text', 'labels']]

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24200 entries, 0 to 24199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    24200 non-null  object 
 1   labels  24200 non-null  float64
dtypes: float64(1), object(1)
memory usage: 378.2+ KB


In [ ]:
train_data['labels'] = train_data['labels'].astype(int)

In [ ]:
x_data = train_data['text']
y_data = train_data['labels']

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.2, stratify=y_data, random_state=43)

In [ ]:
valid_data = pd.DataFrame({'text': x_valid, 'labels': y_valid})

In [ ]:
train_data = pd.DataFrame({'text': x_train, 'labels': y_train})

In [ ]:
valid_data['labels'] = valid_data['labels'].astype(int)

In [ ]:
valid_data.count()

text      4840
labels    4840
dtype: int64

In [ ]:
import torch 
from transformers import BertTokenizer

In [ ]:
torch.cuda.empty_cache()
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

#If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A100-SXM4-40GB


In [ ]:
#Loading locally the bert model
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")

In [ ]:
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, random_split

In [ ]:
from datasets import Dataset

In [ ]:
train_dataset = Dataset.from_pandas(train_data)

In [ ]:
valid_dataset = Dataset.from_pandas(valid_data)

In [ ]:
def processingToken(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return bert_tokenizer(batch["text"], max_length=512, truncation=True, padding="max_length")

In [ ]:
# Encode the input data
train_dataset = train_dataset.map(processingToken, batched=True)
# Transform to pytorch tensors and only output the required columns
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

  0%|          | 0/20 [00:00<?, ?ba/s]

In [ ]:
# Encode the input data
valid_dataset = valid_dataset.map(processingToken, batched=True)
# Transform to pytorch tensors and only output the required columns
valid_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

  0%|          | 0/5 [00:00<?, ?ba/s]

In [ ]:
valid_dataset = valid_dataset.remove_columns('__index_level_0__')

In [ ]:
from transformers import BertConfig, BertAdapterModel

In [ ]:
config = BertConfig.from_pretrained(
    "bert-base-multilingual-uncased",
    num_labels=2,    
)

In [ ]:
model = BertAdapterModel.from_pretrained(
    "bert-base-multilingual-uncased",
    config=config,
)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertAdapterModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Add a new adapter
model.add_adapter("sent_ana_port", config="pfeiffer")
# Add a matching classification head
model.add_classification_head(
    "sent_ana_port",
    num_labels=2, 
  )
# Activate the adapter
model.train_adapter("sent_ana_port")

In [ ]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

In [ ]:
training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [ ]:
def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

In [ ]:
trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_accuracy,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 19360
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7260


Step,Training Loss
200,0.270200
400,0.188700
600,0.175600
800,0.183600
1000,0.163300
1200,0.165600
1400,0.167100
1600,0.146000
1800,0.144500
2000,0.145700


Saving model checkpoint to ./training_output/checkpoint-500
Configuration saved in ./training_output/checkpoint-500/sent_ana_port/adapter_config.json
Module weights saved in ./training_output/checkpoint-500/sent_ana_port/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-500/sent_ana_port/head_config.json
Module weights saved in ./training_output/checkpoint-500/sent_ana_port/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-500/sent_ana_port/head_config.json
Module weights saved in ./training_output/checkpoint-500/sent_ana_port/pytorch_model_head.bin
Saving model checkpoint to ./training_output/checkpoint-1000
Configuration saved in ./training_output/checkpoint-1000/sent_ana_port/adapter_config.json
Module weights saved in ./training_output/checkpoint-1000/sent_ana_port/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-1000/sent_ana_port/head_config.json
Module weights saved in ./training_output/checkpoint-1000/sent_an

TrainOutput(global_step=7260, training_loss=0.1410400807036513, metrics={'train_runtime': 2012.0148, 'train_samples_per_second': 57.733, 'train_steps_per_second': 3.608, 'total_flos': 3.109293563904e+16, 'train_loss': 0.1410400807036513, 'epoch': 6.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 4840
  Batch size = 16


{'eval_loss': 0.15434417128562927,
 'eval_acc': 0.9409090909090909,
 'eval_runtime': 37.9378,
 'eval_samples_per_second': 127.577,
 'eval_steps_per_second': 7.987,
 'epoch': 6.0}

In [ ]:
model.save_adapter("./final_adapter", "sent_ana_port")

!ls -lh final_adapter

Configuration saved in ./final_adapter/adapter_config.json
Module weights saved in ./final_adapter/pytorch_adapter.bin
Configuration saved in ./final_adapter/head_config.json
Module weights saved in ./final_adapter/pytorch_model_head.bin


total 5.7M
-rw-r--r-- 1 root root 1.1K Jan 25 23:30 adapter_config.json
-rw-r--r-- 1 root root  417 Jan 25 23:30 head_config.json
-rw-r--r-- 1 root root 3.5M Jan 25 23:30 pytorch_adapter.bin
-rw-r--r-- 1 root root 2.3M Jan 25 23:30 pytorch_model_head.bin
